### 청원목록 크롤러

In [1]:
!python -m pip install --upgrade pip
!pip install html5lib beautifulsoup4   # HTML을 받아오기 위한 설치 작업

from urllib import request   # HTML을 받아오기 위한 import
from bs4 import BeautifulSoup  # HTML 문자열을 분석하여 DOM을 구성하기 위한 import

import pandas as pd   # 데이터프레임 처리/분석을 위한 import

  Using cached https://files.pythonhosted.org/packages/5f/25/e52d3f31441505a5f3af41213346e5b6c221c9e086a166f3703d2ddaf940/pip-18.0-py2.py3-none-any.whl
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:


Could not install packages due to an EnvironmentError: [WinError 5] 액세스가 거부되었습니다: 'c:\\programdata\\anaconda3\\lib\\site-packages\\pip\\_internal\\basecommand.py'
Consider using the `--user` option or check the permissions.

You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
Invalid requirement: '#'
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\packaging\requirements.py", line 93, in __init__
    req = REQUIREMENT.parseString(requirement_string)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1632, in parseString
    raise exc
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1622, in parseString
    loc, tokens = self._parse( instring, 0 )
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1379, in _parseNoCache
    loc,tokens = self.

In [2]:
# 번호, 분류, 제목, 참여인원을 불러오면 앞뒤로 붙어있는 텍스트를 지우기 위한 함수.
# 참여인원은 "명"이 추가로 붙어 있으니 참여인원할 때 추가 작업이 필요.

# '>' 라는 문자를 기준으로 자르고(split), 자른 결과의 4번째('[3]') 요소를 선택해서,
# 다시 '<' 라는 문자 기준으로 자른 다음 1번째('[0]') 요소를 선택하여 출력한다.

def to_return(raw):
  result = raw.split(">")[3].split("<")[0]
  return result

In [3]:
# 번호 : to_return 함수를 호출해서 bl_no의 앞뒤 텍스트를 지우고 no이라는 리스트에 넣는다.

def return_no(bs):

  bl_no = bs.select('.wrap > .board.text .bl_body ul li .bl_wrap .bl_no')

  no = []
  i_no = 0   # bl_no가 리스트 타입이기 때문에 1개씩 불러오려면 인덱스가 필요하다.

  for num in bl_no:   # bl_no를 1개씩 순차적으로 불러오기 위한 반복문
    num = str(bl_no[i_no])   # bl_no의 요소는 nontype → str으로 바꿔야 to_return 함수에서 split가 가능
    no.append(int(to_return(num)))   # to_return의 결과값은 str이므로 int로 변경해서 저장
    i_no = i_no + 1
  
  return no

In [4]:
# 분류 : to_return 함수를 호출해서 bl_category의 앞뒤 텍스트를 지우고 category라는 리스트에 넣는다.

def return_category(bs):
  
  bl_category = bs.select('.wrap > .board.text .bl_body ul li .bl_wrap .bl_category.cs')

  category = []
  i_cate = 0

  for catego in bl_category:
    catego = str(bl_category[i_cate])
    category.append(to_return(catego))
    i_cate = i_cate + 1
  
  return category

In [5]:
# 참여인원 : to_return 함수를 호출해서 bl_agree의 앞뒤 텍스트를 지우고, 
# 추가 작업을 한 후 num_agree라는 리스트에 넣는다.

def return_agree(bs):

  # 'bl_agree_cb'가 대부분이지만 'bl_agree_cs'가 간혹 있음 : 'bl_agree'로 읽으면 모두 불려옴
  bl_agree = bs.select('.wrap > .board.text .bl_body ul li .bl_wrap .bl_agree')
  
  num_agree = []
  i_agree = 0

  for agree in bl_agree:
    agree = str(bl_agree[i_agree])
  
    # 참여인원은 글 번호와 다르게 숫자 뒤에 "명"과 빈 칸이 붙어 있어서 지워야 함.
    no_agree = to_return(agree).split("명")[0]
  
    num_agree.append(no_agree)
    i_agree = i_agree + 1

  return num_agree

In [6]:
# 제목 : to_return 함수를 호출해서 bl_subject의 앞뒤 텍스트를 지우고 subject라는 리스트에 넣는다.

def return_sub(bs):

  bl_subject = bs.select('.wrap > .board.text .bl_body ul li .bl_wrap .bl_subject a')

  subject = []
  i_subject = 0

  for sub in bl_subject:
    sub = str(bl_subject[i_subject])
    subject.append(to_return(sub))
    i_subject = i_subject + 1

  return subject

In [7]:
# 데이터프레임에 데이터를 추가하는 함수

def return_df(page):
  
  # HTML 받아오기
  
  with request.urlopen(page) as f:
    charset = f.headers.get_content_charset()
    html = f.read().decode(charset)
   
  # HTML 문자열을 분석하여 DOM 구성하기

  bs = BeautifulSoup(html, 'html5lib')

  # 각 페이지의 번호, 분류, 제목, 참여인원 데이터를 뽑아 데이터프레임에 추가한다.

  no = return_no(bs)
  category = return_category(bs)
  subject = return_sub(bs)
  num_agree = return_agree(bs)
  
  # 크롤링 시 array 문제가 생기면, 오류 발생 위치를 확인하기 위한 코드 
  # print(len(no), len(category), len(subject), len(num_agree), int(i))
  
  option_list = {'No': no, 'Category': category, 'Subject': subject, 'Agree': num_agree}
  df_pg = pd.DataFrame(option_list, columns = ['No', 'Category', 'Subject', 'Agree'])
  
  return df_pg

In [8]:
# 페이지 주소 자동 생성

def page_url(pg_num):
  page = "https://www1.president.go.kr/petitions?page=" + str(pg_num)
  return page

In [9]:
# 청원 목록 한 페이지의 게시물 수는?
df_leng_of_page = return_df(page_url(0))
leng_of_page = len(df_leng_of_page.index)

# 현재 시점 기준 청원 목록의 마지막 번호는?
last_page_url = page_url(1)
last_page = return_df(last_page_url)

# 청원 목록의 페이지 수는? (= 마지막 번호 / 한 페이지 게시물 수)
pg_no = int(int(last_page.iloc[0,0]) / leng_of_page)

# 청원 목록의 페이지 번호는 1번에서 시작하지만, 파이썬 인덱스는 0에서 시작한다.
# 그래서, 실제 마지막 페이지 번호는 1만큼 더해줘야 한다.
pg_num = pg_no + 1

pg_num

16579

In [10]:
data_petitions = pd.DataFrame()   # 데이터를 추가할 빈 그릇을 하나 만든다.
i = 1   # for문으로 반복시키기 위한 index 번호도 한 개 만든다.


# 청원목록 전체 크롤링. (주의 : 오래 걸림 / 기준 시점 : 20180728 PM 11:37)
for i in range (pg_num):

# for i in range (15):  

  data_pg = return_df(page_url(i))
  
  if i == 0:
    data_list = data_petitions.append(data_pg, ignore_index = True)

  else:
    data_list = data_list.append(data_pg, ignore_index = True)

  i = i + 1
    
data_list

URLError: <urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>

In [ ]:
# 분류와 제목을 기준으로 중복된 행을 제거한다. 
# 도배 게시물을 제거하여 분석의 정확성을 높이는 목적

list_result = data_list.drop_duplicates(["Category", "Subject"], keep="first")

In [ ]:
# 청원목록 전체 크롤링 자료에 대한 작업용 복사본 생성
    
petitions_all_list = list_result

In [ ]:
# 크롤링 완료된 자료를 Excel에 저장

list_result.to_excel('petitions_data.xlsx', sheet_name='list')

In [ ]:
# Excel에 저장한 자료를 읽어올 수 있음

pd.read_excel('petitions_data.xlsx', 'list', index_col=None, na_values=['NA'])

### 청원 본문 크롤러

In [ ]:
!python -m pip install --upgrade pip
!pip install html5lib beautifulsoup4   # HTML을 받아오기 위한 설치 작업
!pip install -q 'soynlp[all]'   # 자연어 처리/분석을 위한 soynlp 설치 작업
!pip show soynlp   # 자연어 처리/분석을 위한 soynlp 설치 작업 2

from urllib import request   # HTML을 받아오기 위한 import
from bs4 import BeautifulSoup   # HTML 문자열을 분석하여 DOM을 구성하기 위한 import

import pandas as pd   # 데이터프레임 처리/분석을 위한 import
import numpy as np   # 데이터프레임 처리/분석을 위한 import 2
import re   # 정규표현식 사용을 위한 import
import random   # Random 선택 사용을 위한 import

In [ ]:
# 제목을 불러오면 앞뒤로 붙어있는 텍스트를 지우기 위한 함수.

# '>' 라는 문자를 기준으로 자르고(split), 자른 결과의 4번째('[1]') 요소를 선택해서,
# 다시 '<' 라는 문자 기준으로 자른 다음 1번째('[0]') 요소를 선택하여 출력한다.

def to_cleaning(raw):
  result = raw.split(">")[1].split("<")[0]
  return result

In [ ]:
# 제목을 불러오는 함수

def return_title(bs_contents):

  ti = bs_contents.select('.petitionsView_left_pg .petitionsView_title')
  #title = ti[0]

  title = []   # "ValueError: If using all scalar values, you must pass an index site" 방지 : 리스트로 변환

  title_text = str(ti)   # ti를 str으로 바꿔서 to_cleaning 함수에서 split가 가능하도록 만듦
  title.append(to_cleaning(title_text))
  
  return title

In [ ]:
# 본문 : 개행문자 / HTML 코드 제거를 위한 함수

def preprocessing_con(con_clear):
  con_clear = re.sub('\t', '', con_clear)
  con_clear = re.sub('\n', '', con_clear)
  con_clear = re.sub('<br>', '', con_clear)
  con_clear = re.sub('<br/>', '', con_clear)
  con_clear = re.sub('<div class="View_write" style="word-break:break-all">', '', con_clear)
  con_clear = re.sub('&lt;', '', con_clear)
  con_clear = re.sub('&gt;', '', con_clear)
  con_clear = re.sub('</div>', '', con_clear) 
  con_clear = re.sub('\[', '', con_clear)
  con_clear = re.sub('\]', '', con_clear)  
  return con_clear

In [ ]:
# 본문을 불러오는 함수 : 개행문자 제거를 위해 preprocessing_con 함수를 호출함

def return_contents(bs_contents):

  con = bs_contents.select('.petitionsView_left_pg .petitionsView_write .View_write')
  
  contents = []   # "ValueError: If using all scalar values, you must pass an index site" 방지 : 리스트로 변환

  contents_text = str(con)   # con을 str으로 바꿔서 to_cleaning 함수에서 split가 가능하도록 만듦
  contents.append(preprocessing_con(contents_text))
   
  return contents

In [ ]:
# 페이지 주소 자동 생성

def page_url_con(pg_con):
  page_contents = "https://www1.president.go.kr/petitions/" + str(pg_con)
  return page_contents

In [ ]:
# 데이터프레임에 데이터를 추가하는 함수

def return_df_contents(page_contents):
  
  # HTML 받아오기
  
  with request.urlopen(page_contents) as f:
    charset = f.headers.get_content_charset()
    html = f.read().decode(charset)
   
  # HTML 문자열을 분석하여 DOM 구성하기

  bs_contents = BeautifulSoup(html, 'html5lib')

  # 각 페이지의 제목, 본문 데이터를 뽑아 데이터프레임에 추가한다.

  title = return_title(bs_contents)
  contents = return_contents(bs_contents)
  
  # 크롤링 시 array 문제가 생기면, 오류 발생 위치를 확인하기 위한 코드 
  # a = print(len(title), len(contents), int(i))
  
  option_list_con = {'Title': title, 'Contents': contents}
  df_contents = pd.DataFrame(option_list_con, columns = ['Title', 'Contents'])
  
  return df_contents

In [ ]:
# Q. 현재 시점 기준 청원 목록의 마지막 번호에 해당하는 본문 페이지의 번호는?
# A. 청원목록 1페이지의 가장 최신글 제목에 설정되어 있는 링크를 가져오면 된다.

# 우선 청원목록 1페이지를 가져온다.
page_last = "https://www1.president.go.kr/petitions?page=1"

# 청원목록 1페이지의 HTML을 받아온 후,
with request.urlopen(page_last) as f:
    charset = f.headers.get_content_charset()
    html = f.read().decode(charset)

# 청원목록 1페이지의 HTML DOM을 구성한다.
last_pg = BeautifulSoup(html, 'html5lib')

# 제목 부분만 뽑아온 후,
bl_last = last_pg.select('.wrap > .board.text .bl_body ul li .bl_wrap .bl_subject a')

# 제목 중에서도 가장 최신글만 선택해서,
last = str(bl_last[0])

# 숫자 부분만 split로 뽑아낸 다음, 
last_num = last.split("\"")[3].split("/")[4].split("?")[0]

# str으로 split 작업한 결과물을 int로 변경한다.
last_num_int = int(last_num)

last_num_int

In [ ]:
data_petitions_contents = pd.DataFrame()   # 데이터를 추가할 빈 그릇을 하나 만든다.

# for문으로 반복시키기 위한 index 번호도 한 개 만든다.
# 청와대 국민청원 본문 페이지는 21번부터 시작하므로 index도 21번부터 시작해야 한다.
i = 21

# for문으로 반복시킬 때, exception 처리 되지 않은 첫 번째 결과값에 대해 빈 그릇을 적용해야 한다.
# exception 처리 되지 않은 첫 번째 결과값 여부를 판단하기 위한 index를 한 개 만든다.
try_num = 0

# 청원 본문 전체 크롤링. (주의 : 오래 걸림 / 기준 시점 : 20180728 PM 3:00)
# for i in range (21, last_num_int): (전체 크롤링은 너무 오래 걸려서 잠깐 봉인...)

for i in range (313457, 313459):

  while True:
    try:
      data_content = return_df_contents(page_url_con(i))
     
    except Exception:
      continue

  try_num = try_num + 1
  
  if try_num == 1:
    data_list_contents = data_petitions_contents.append(data_content, ignore_index = True)
    
  else:
    data_list_contents = data_list_contents.append(data_content, ignore_index = True)
    
  i = i + 1
  
  a = data_list_contents
    
a

In [ ]:
# 제목을 기준으로 중복된 행을 제거한다. 
# 도배 게시물을 제거하여 분석의 정확성을 높이는 목적

contents_result = data_list_contents.drop_duplicates(["Title"], keep="first")

In [ ]:
# 청원목록 전체 크롤링 자료에 대한 작업용 복사본 생성
    
petitions_all_contents = contents_result

In [ ]:
# 크롤링 완료된 자료를 Excel에 저장

contents_result.to_excel('petitions_data.xlsx', sheet_name='contents')

In [ ]:
# Excel에 저장한 자료를 읽어올 수 있음

pd.read_excel('petitions_data.xlsx', 'contents', index_col=None, na_values=['NA'])